The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [13]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd    
from model import initialize_model, call_llm
import time
import json



/Users/jasonchung/Downloads/nlp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [3]:
data = preprocess("./data/train.jsonl")

In [4]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [6]:
labels = []
reasonings = []
raw_output = []
ids = []
client, _ = initialize_model("")
last_time = time.time()

# Change len(data) to len(your_partition)
for i in range(len(sixth_partition)):
    if i > 0 and i % 60 == 0:
        print(i)
        current_time = time.time()
        time.sleep(max(0, 63 - (current_time - last_time)))
        last_time = time.time()

    # Change data to your partition
    current_data = sixth_partition.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_llm(client, "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", current_data.sectionName, current_data.string)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)
    

id='noTSe5z-4Yz4kd-929efb407ea8409a' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743582807 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=16722161866320394000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='{"classification": "background", "reasoning": "The text provides historical context and references previous studies (Vander Heiden et al., 2000, 2001) to establish the role of BCL-xL in regulating metabolite flux, which is a common characteristic of background sections in scientific papers that aim to introduce and contextualize the research topic."}', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=199, completion_tokens=75, total_tokens=274, cached_tokens=0)
id='noTSjBJ-4Yz4kd-929efbab9da8409a' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743582824 model='meta-llama/Llama-3.3-70B-In

In [15]:
print(sixth_partition)

        source  citeEnd           sectionName  citeStart  \
6860  explicit    233.0               Results      208.0   
6861  explicit    176.0         7 EXPERIMENTS      172.0   
6862  explicit    210.0  3.2.2 Implementation      206.0   
6863  explicit      8.0             6 OUTLOOK        3.0   
6864  explicit    184.0            Discussion      164.0   
...        ...      ...                   ...        ...   
8238  explicit     50.0                             28.0   
8239  explicit    182.0            DISCUSSION      179.0   
8240  explicit    120.0            DISCUSSION      108.0   
8241  explicit    221.0                            185.0   
8242  explicit    102.0          Introduction       82.0   

                                                 string       label  \
6860  Work with non-neuronal cells has suggested tha...  background   
6861  We have added the affordances to the definitio...      method   
6862  To determine this canonical set of values, the...      metho

In [8]:
filename = "sixth_corrected.txt"
c = []
r = []
i = [sixth_partition.iloc[x].unique_id for x in range(len(sixth_partition))]

with open(filename) as file:
    lines = [line.rstrip() for line in file]
    for line in lines:
        start = line.find("{")
        end = line.rfind("}") + 1
        if start == -1:
            continue
        json_str = json.dumps(eval(line[start:end]))
        
        answers = json.loads(json_str)
        
        classification = answers["classification"]
        reasoning = answers["reasoning"]
        print(classification)
        print(reasoning)
        c.append(classification)
        r.append(reasoning)

df = pd.DataFrame(zip(i, c, r), columns=["id", "model_classification", "reasoning"])
df.to_csv("your_partition.csv")

background
Although the section name is 'Results', the text itself discusses previous research and established knowledge, citing specific studies from 2000 and 2001, which is characteristic of a background section. It does not present new findings or data, but rather provides context and foundation for the current study.
method
The text discusses the process of adding affordances to definitions and potential methods for automating this process, such as using semantic annotations or geometric reasoning, which is typical of a methods section in a scientific paper.
method
The text describes the process of determining a canonical set of values and mentions the use of parameters from a mathematical model, indicating a step-by-step approach to achieving a specific goal, which is typical of a method section in a scientific paper.
background
The text mentions a specific reference ([103]) and describes a development in the field, indicating that it is providing context or background information

In [7]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("sixth_partition_llama_no_section_name.csv")

In [15]:
part = pd.read_csv("./results/meta-llama_Llama-3.3-70B-Instruct-Turbo-Free/sixth_partition.csv")

In [16]:
part = part.rename(columns={"id": "id", "classification": "model_classification", "reasoning": "reasoning"})

In [17]:
print(part.head())

   Unnamed: 0                                                 id  \
0           0  27db8d3a2c85cbc34deef51ce4cd850b431b4b34>a81b3...   
1           1  45389b607544d07ecb6d94e3a4ae3da794b0bb77>84c2d...   
2           2  55294c3cfc8a43921fb1a604c21d33f524a64927>28969...   
3           3  058bf8bc9f753b371e340ee08afec4b41752ed67>c2cec...   
4           4  0a9889f9add7c40c6295c7573feab8bd13662583>1b176...   

  model_classification                                          reasoning  
0           background  Although the section name is 'Results', the te...  
1               method  The text mentions 'EXPERIMENTS' which is a com...  
2               method  The text describes the implementation of a spe...  
3           background  The text is classified as background because i...  
4           background  The text mentions a specific study (Adesnik et...  


In [18]:
part = part.drop(columns=['Unnamed: 0'])


In [19]:
part.to_csv("sixth_partition.csv")